In [453]:
import json
import random
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
import pandas as pd
import numpy as np
import seaborn as sns
from scipy import spatial

import pandas as pd
import numpy as np

import os
import datetime
import string
import nltk

In [454]:
# df = pd.read_csv('scraper results/data scientist.csv')
# df

In [455]:
# df.dtypes

In [456]:
wordnet_lemmatizer = WordNetLemmatizer()

#defining the function to remove punctuation
def remove_punctuation(text):
    punctuationfree="".join([i for i in text if i not in string.punctuation])
    return punctuationfree

#defining the function for lemmatization
def lemmatizer(text):
    lemm_text = [wordnet_lemmatizer.lemmatize(word) for word in text]
    return " ".join(lemm_text)

#Stop words present in the library
stopwords = nltk.corpus.stopwords.words('english')

#defining the function to remove stopwords from tokenized text
def remove_stopwords(text):
    output = [i for i in "".join(text).split(" ") if i not in stopwords]
    return " ".join(output)


def clean_job_description(df: pd.DataFrame):
    df['job_description_cleaned'] = df['Job Description'].apply(lambda x:remove_punctuation(x))
    df['job_description_cleaned'] = df['job_description_cleaned'].apply(lambda x: x.lower())

    df['job_description_cleaned'] = df['job_description_cleaned'].apply(lambda x:remove_stopwords(x))
    df['job_description_cleaned'] = df['job_description_cleaned'].str.replace(r"(\n)+"," ", regex = True)

    df['job_description_cleaned'] = df['job_description_cleaned'].apply(lambda x:lemmatizer(x.split(" ")))


def get_word_counts_df(df):
    temp = df["job_description_cleaned"].str.split(expand=True).stack().value_counts()
    ret = pd.DataFrame(temp, columns=["count"])
    return ret

In [457]:
# df = pd.read_csv('data_jobs_data/data/DataScientist.csv').drop(['Unnamed: 0', 'index'],axis=1)
# # df = pd.read_csv('scraper results/data scientist 900.csv')
# df

In [458]:
def remove_duplicates(df:pd.DataFrame, exclude=None):
    
    subset_columns = ['Job Title', 'Company Name', 'Location', 'Salary Estimate', 'Rating', 'Job Description']
    
    if exclude:
        subset_columns.remove(exclude)
    
    df.sort_values(subset_columns, inplace=True)

    df['Duplicate Count'] = df.groupby(subset_columns)['Job Title'].transform('count')
    df.drop_duplicates(subset=subset_columns, inplace=True, keep='last')
    

In [ ]:
['Job Title', 'Company Name', 'Location', 'Salary Estimate', 'Rating', 'Job Description', 'Founded', 'Industry', 'Revenue', 'Sector', 'Size', 'Type', 'Easy Apply']

In [459]:

# Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,Industry,Sector,Revenue,Competitors,Easy Apply
def convert_sqft_to_num(x,agg):
    if x == '-1' or not x or x == np.NaN:
        return None
    tokens = x.split('-')

    try:
        if len(tokens) == 2:
            if agg == 'mean':
                return (float(tokens[0])+float(tokens[1]))/2
            elif agg == 'min':
                return min(float(tokens[0]),float(tokens[1]))
            elif agg == 'max':
                return max(float(tokens[0]),float(tokens[1]))
        return float(x)
    except Exception as e:
        print(tokens)
        print(e)
        return None

def isrevenue(x):
    if x=="Non-Applicable":
        return 0
    elif x=="(USD)":
        return 1

def salary_type(s):
    if not isinstance(s,str):
        return None
    if 'Per Hour' in s:
        return 'Per Hour'
    else:
        return 'Annual (K)'

def level(s):
    if 'Senior' in s:
        return 


def reorder_dataframe_columns_inplace(df:pd.DataFrame, desired_order, prefix='delete_'):
    filtered_order = [col for col in desired_order if col in df.columns]
    df.rename(columns={col: prefix + col for col in filtered_order}, inplace=True)
    for col in filtered_order:
        df[col] = df[prefix + col]
        del df[prefix + col]

def preprocessing(df: pd.DataFrame):
    df.rename(columns = {'Type of ownership':'Type'}, inplace = True) # ,'Salary Estimate':'salary'
    for col in {'Unnamed: 0', 'index', 'Competitors', 'Headquarters'}:
        if col in df.columns:
            df.drop(col, axis=1, inplace=True)

    reorder_dataframe_columns_inplace(df, ['Job Title', 'Company Name', 'Location', 'Salary Estimate', 'Rating', 'Job Description', 'Founded', 
                                           'Industry', 'Revenue', 'Sector', 'Size', 'Type', 'Easy Apply', 'Duplicate Count'])

    df['Job Title clean'] = df['Job Title'].apply(lambda x:x.join(x.split(',')[:1]))
    df['Job Title clean'] = df['Job Title clean'].apply(lambda x:x.join(x.split('/')[:1]))
    df['Job Title clean'] = df['Job Title clean'].apply(lambda x:x.join(x.split(' - ')[:1]))
    df['Job Title clean'] = df['Job Title clean'].str.replace('Sr.', 'Senior')
    df['Job Title clean'] = df['Job Title clean'].str.replace('Sr', 'Senior')
    df['Job Title clean'] = df['Job Title clean'].str.replace('Jr.', 'Junior')
    df['Job Title clean'] = df['Job Title clean'].str.replace('Jr', 'Junior')

    df['Job Title clean'] = df['Job Title clean'].apply(lambda x:x.replace('#NAME?','-'))
    df['Job Title clean'] = df['Job Title clean'].apply(lambda x:x.replace('with','-'))

    if df['Easy Apply'].dtype != bool:
        df['Easy Apply'] = df['Easy Apply'].replace(['-1'],0).replace(['FALSE'],1)
        df['Easy Apply'] = df['Easy Apply'].astype(bool)


    # df.dropna(how='any',axis=0,inplace=True)
    df.fillna(-1,inplace=True)
    df.reset_index(drop=True,inplace=True)

    df['Founded']=df['Founded'].astype(int)

    df['Size'] = df['Size'].apply(lambda x:x.replace('to','-'))
    df['Size'] = df['Size'].apply(lambda x:x.replace('+',' - 0'))
    df['Size'] = df['Size'].apply(lambda x:x.replace('employees',' '))
    df['Size'] = df['Size'].apply(lambda x:x.replace('Employees',' ')).str.replace(' ','')

    df['Salary Type'] = df['Salary Estimate'].apply(salary_type)


    # # Drop the rows where salary missing
    # df.drop(df[df['Salary Estimate'].isna() | (df['Salary Estimate'] == '-1')].index, inplace=True)

    # df['Salary Estimate'].replace([-1, None], '-1')
    df['Salary Estimate'] = df['Salary Estimate'].str.replace('[^0-9\-]', '', regex=True)
    df['Salary Estimate'].fillna('-1',inplace=True)

    df['min_salary'] = df['Salary Estimate'].apply(lambda x: convert_sqft_to_num(x, 'min'))
    df['max_salary'] = df['Salary Estimate'].apply(lambda x: convert_sqft_to_num(x, 'max'))

    # replace -1 with nan
    df['Revenue'] = df['Revenue'].replace(['-1', 'Unknown / Non-Applicable'],None)
    df['Sector'] = df['Sector'].replace(['-1'],None)
    df['Type'] = df['Type'].replace(['-1', 'Unknown'],None)
    df['Size'] = df['Size'].replace(['-1', 'Unknown'],None)
    df['Founded'] = df['Founded'].replace([-1],None)
    df['Rating'] = df['Rating'].replace([-1],None)


def extract_years_experience(description):
    # Updated regex pattern to match numbers with an optional '+' sign followed by 'year' or 'years'
    # This will match '2 years', '5-7 years', '3+ years', '3 year', etc.
    pattern = r'(\d+\+?|\d+-\d+)\s+years?'

    # Search for the pattern in the description
    match = re.search(pattern, description.lower())
    if match:
        return match.group(1)
    else:
        return None




def clean_data(df):
    print(df.shape)
    remove_duplicates(df)
    print(df.shape)
    preprocessing(df)
    print(df.shape)
    
    clean_job_description(df)
    

    # Apply the function to create a new column
    df['Years Experience'] = df['Job Description'].apply(extract_years_experience)

    
    return df

In [460]:
df = pd.read_csv('data_jobs_data/data/DataEngineer.csv')
# df
clean_data(df)
df.to_csv('data_jobs_data/data/DataEngineer_clean.csv',index=False)
df

(2528, 15)
(2516, 16)
(2516, 18)


,Job Title,Company Name,Location,Salary Estimate,Rating,Job Description,Founded,Industry,Revenue,Sector,Size,Type,Easy Apply,Duplicate Count,Job Title clean,Salary Type,min_salary,max_salary,job_description_cleaned,Years Experience
0,!!!100% Remote!!! Sr. Data and Integration Engineer,Amerit Consulting\n3.5,"Dallas, TX",74-134,3.5,"Job Description\nOur client is an American for-profit managed health care company, focused on sp...",2002,State & Regional Agencies,$5 to $10 million (USD),Government,51-200,Company - Private,False,1,!!!100% Remote!!! Senior Data and Integration Engineer,Annual (K),74.0,134.0,job description our client american forprofit managed health care company focused special popula...,None
1,#104733 HPC Systems and Data Engineer,UC San Diego\n4.3,"San Diego, CA",46-92,4.3,The effects of the COVID-19 pandemic have impacted UC San Diego Campus hiring. UCSD Campus will ...,1960,Colleges & Universities,None,Education,10000-0,College / University,False,1,#104733 HPC Systems and Data Engineer,Annual (K),46.0,92.0,effect covid19 pandemic impacted uc san diego campus hiring ucsd campus continue recruit essenti...,None
2,(Big Data Developer) Applications Systems Engineer 4,Wells Fargo\n3.6,"Chandler, AZ",66-122,3.6,"Job Description\n\nImportant Note: During the application process, ensure your contact informati...",1852,Banks & Credit Unions,$10+ billion (USD),Finance,10000-0,Company - Public,False,1,(Big Data Developer) Applications Systems Engineer 4,Annual (K),66.0,122.0,job description important note application process ensure contact information email phone number...,5+
3,"(Project Staff) Consultant, Data Engineer",Nationwide\n3.6,"Scottsdale, AZ",81-141,3.6,"As a team member in the Finance and Internal Audit department at Nationwide, the opportunities a...",1925,-1,$10+ billion (USD),None,10000-0,Company - Private,False,1,(Project Staff) Consultant,Annual (K),81.0,141.0,team member finance internal audit department nationwide opportunity endless grow learn diverse ...,None
4,"(SONUS) Engineer/Senior Engineer, IT Network Infrastructure",American Airlines\n3.8,"Fort Worth, TX",60-109,3.8,"(SONUS) Engineer/Senior Engineer, IT Network Infrastructure\n\n\n\n(SONUS) Engineer/Senior Engin...",1926,Airlines,$10+ billion (USD),Travel & Tourism,10000-0,Company - Public,False,1,(SONUS) Engineer,Annual (K),60.0,109.0,sonus engineersenior engineer network infrastructure sonus engineersenior engineer network infra...,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2511,Wireless Solutions Engineer - Sales Analytics,Crown Castle\n3.6,"Houston, TX",54-113,3.6,Position Summary\n\nAs a Wireless Solutions Engineer you will use your advanced data analytics a...,1994,Telecommunications Services,$2 to $5 billion (USD),Telecommunications,1001-5000,Company - Public,False,1,Wireless Solutions Engineer,Annual (K),54.0,113.0,position summary a wireless solution engineer use advanced data analytics rf network engineering...,3-5
2512,Wiring Installation Engineer,Butler Technical Group\n3.8,"Fort Worth, TX",56-89,3.8,"Wiring Installation Engineer\n\nLocation: Fort Worth, TX\n\nDescription\n\nThe Wiring Installati...",None,-1,None,None,501-1000,College / University,False,1,Wiring Installation Engineer,Annual (K),56.0,89.0,wiring installation engineer location fort worth tx description the wiring installation engineer...,5+
2513,eCom Data Engineer,PepsiCo\n3.7,"New York, NY",80-150,3.7,Auto req ID: 212077BR\nJob Description\n\n\nAs other sectors have shifted to eCommerce-first bus...,1965,Food & Beverage Manufacturing,$10+ billion (USD),Manufacturing,10000-0,Company - Public,False,1,eCom Data Engineer,Annual (K),80.0,150.0,auto req id 212077br job description a sector shifted ecommercefirst business model recent year ...,6+
2514,eDiscovery Pre-Sales Engineer 19-0828,Relativity\n3.7,"Chicago, IL",133-210,3.7,"We are Relativity. A market-leading, global tech company that equips legal professionals with a ..."

In [ ]:
for fname in os.listdir('combined scraper results'):
    data = pd.read_csv('combined scraper results/data engineer 2465.csv')
    clean_data(data)
    data.to_csv('combined scraper results/data engineer 2465 clean.csv',index=False)
    print(data)

In [461]:
pd.set_option('display.max_colwidth', 50)
df_new = pd.read_csv('combined scraper results/data engineer 2465.csv')
clean_data(df_new)
df_new.to_csv('combined scraper results/data engineer 2465 clean.csv',index=False)
df_new

(2465, 14)
(2465, 14)
(2157, 18)


,Job Title,Company Name,Location,Salary Estimate,Rating,Job Description,Founded,Industry,Revenue,Sector,Size,Type,Easy Apply,Duplicate Count,Job Title clean,Salary Type,min_salary,max_salary,job_description_cleaned,Years Experience
0,(ESEA) Systems Engineer level 2 (Data Flow and...,"The Kenjya-Trusant Group, LLC\n","Annapolis Junction, MD",59-90,3.7,POSITION: Systems Engineer 2 (Data Flow and Ro...,2002,Business Consulting,$5 to $25 million (USD),Management & Consulting,201-500,Company - Private,True,1.0,(ESEA) Systems Engineer level 2 (Data Flow and...,Annual (K),59.0,90.0,position system engineer 2 data flow routing s...,15
1,(Hybrid) Senior Enterprise Data Engineer,PenFed Credit Union\n,"McLean, VA",101-140,3.5,Overview\n\nAre you looking to take your caree...,1935,Banking & Lending,None,Financial Services,1001-5000,Nonprofit Organization,False,1.0,(Hybrid) Senior Enterprise Data Engineer,Annual (K),101.0,140.0,overview are looking take career good great em...,5+
2,(Hybrid) Senior Enterprise Data Engineer - San...,PenFed Credit Union\n,"San Antonio, TX",80-125,3.5,Overview\n\nAre you looking to take your caree...,1935,Banking & Lending,None,Financial Services,1001-5000,Nonprofit Organization,False,1.0,(Hybrid) Senior Enterprise Data Engineer,Annual (K),80.0,125.0,overview are looking take career good great em...,5+
3,"(USA) Senior, Software Engineer - Data Ventures",Walmart\n,"Dallas, TX",90-180,3.4,Position Summary...\n\nWhat you'll do...\n\nDo...,1962,General Merchandise & Superstores,$10+ billion (USD),Retail & Wholesale,10000-0,Company - Public,False,1.0,(USA) Senior,Annual (K),90.0,180.0,position summary what youll do do boundless en...,5
4,2024 Graduate - Data Scientist/Engineer - Deci...,Johns Hopkins Applied Physics Laboratory (APL)\n,"Laurel, MD",103-143,4.3,Description\n\nAre you passionate about transl...,1942,Aerospace & Defense,$1 to $5 billion (USD),Aerospace & Defense,5001-10000,Nonprofit Organization,False,1.0,2024 Graduate,Annual (K),103.0,143.0,description are passionate translating academi...,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2458,Wire Data Engineer,Tekskills Inc\n,"Fort Worth, TX",76-111,4.3,Job Title : Wire engineer\n\nLocation : Fort W...,2003,Information Technology Support Services,None,Information Technology,501-1000,Company - Private,True,1.0,Wire Data Engineer,Annual (K),76.0,111.0,job title wire engineer location fort worth ...,5
2459,Wire data engineer,Tekskills\n,"Fort Worth, TX",76-111,4.3,Job Title : Wire Data Engineer\n\nLocation : F...,2003,Information Technology Support Services,None,Information Technology,501-1000,Company - Private,True,1.0,Wire data engineer,Annual (K),76.0,111.0,job title wire data engineer location fort w...,5
2461,Workday HR Data & Integration Engineer,DraftKings\n,"Boston, MA",63-93,4.0,We’re defining what it means to build and deli...,2012,Internet & Web Services,None,Information Technology,1001-5000,Company - Public,False,1.0,Workday HR Data & Integration Engineer,Annual (K),63.0,93.0,we’re defining mean build deliver extraordinar...,None
2462,data engineer lead,Starbucks\n,"Seattle, WA",114-153,3.7,Job Summary and Mission\n\nThis position contr...,1971,Restaurants & Cafes,$10+ billion (USD),Restaurants & Food Service,10000-0,Company - Public,False,1.0,data engineer lead,Annual (K),114.0,153.0,job summary mission this position contributes ...,None


In [462]:
print(df.isna().sum())

Job Title                    0
Company Name                 0
Location                     0
Salary Estimate              0
Rating                     233
Job Description              0
Founded                    568
Industry                     0
Revenue                    830
Sector                     373
Size                       181
Type                       148
Easy Apply                   0
Duplicate Count              0
Job Title clean              0
Salary Type                  0
min_salary                   0
max_salary                   0
job_description_cleaned      0
Years Experience           716
dtype: int64


In [463]:
print(df_new.isna().sum())

Job Title                    0
Company Name                 0
Location                     0
Salary Estimate              0
Rating                     171
Job Description              0
Founded                    454
Industry                     0
Revenue                    763
Sector                     273
Size                       171
Type                        64
Easy Apply                   0
Duplicate Count              0
Job Title clean              0
Salary Type                 13
min_salary                  13
max_salary                  13
job_description_cleaned      0
Years Experience           493
dtype: int64


In [464]:
df.loc[df.min_salary.isna()].shape

(0, 20)

In [465]:
df_new.loc[df_new.min_salary.isna()].shape

(13, 20)

In [466]:
# print('unique job titles are :',df['Job Title clean'].unique())
print(len(df['Job Title'].unique()),'unique job titles')
print(len(df['Job Title clean'].unique()),'unique job titles (clean)')
sorted(df['Job Title clean'].unique())

1260 unique job titles
972 unique job titles (clean)


['!!!100% Remote!!! Senior Data and Integration Engineer',
 '#104733 HPC Systems and Data Engineer',
 '(Big Data Developer) Applications Systems Engineer 4',
 '(Project Staff) Consultant',
 '(SONUS) Engineer',
 '(Senior) Data Engineer',
 '+ Data Engineer',
 '. Java Engineer',
 '7232 Data Engineer (Analyst',
 ': Senior Data Engineer',
 'ADAS Test V&V Engineer',
 'AI',
 'AI Engineer',
 'AI ML Engineer',
 'AI OCR Engineer (Ony W2)',
 'AI Researcher',
 'AM',
 'AMS Verification and Modeling Engineer',
 'AV ENGINEER',
 'AVP',
 'AWS',
 'AWS Cloud Engineer',
 'AWS Data Engineer',
 'AWS Data Platform Engineer',
 'AWS Engineer',
 'AZURE DATA ENGINEER (Data Bricks)',
 'Ab Initio Engineer',
 'Advanced AI Engineer',
 'Advanced Python Data Engineer',
 'Advanced Software Engineer',
 'Advisory Services',
 'Aerospace Engineer',
 'Aircraft Configuration Engineer',
 'Airflow Data Engineer',
 'Airframe Structural Analysis Engineer-(Ft. Worth or Wichita)',
 'Amazon- Software Development Engineer',
 'Analog

In [467]:
# print('unique job titles are :',df['Job Title clean'].unique())
print(len(df_new['Job Title'].unique()),'unique job titles')
print(len(df_new['Job Title clean'].unique()),'unique job titles (clean)')
sorted(df_new['Job Title clean'].unique())

1229 unique job titles
894 unique job titles (clean)


['(ESEA) Systems Engineer level 2 (Data Flow and Routing Solutions) (TS',
 '(Hybrid) Senior Enterprise Data Engineer',
 '(USA) Senior',
 '2024 Graduate',
 '2025859 Data Scientist (Data Engineer) $215',
 '5+ Years in VOICE',
 '513th Electronic Warfare Mission Data Engineer',
 '5G Data Transport Engineer',
 'AI',
 'AI Engineer',
 'AI Engineer ',
 'AI Operations and Data Transformation Engineer',
 'ASG Data Engineer',
 'ASSETS',
 'AVP Enterprise MS SQL Data Engineer',
 'AWS ',
 'AWS Certified Data Engineer',
 'AWS Cloud Data Engineer & Delivery (REMOTE)',
 'AWS Cloud Data Lead Software Engineer',
 'AWS Data Engineer',
 'AWS Data Engineer ( Hybrid at Eagan',
 'AWS Data Engineer - API Hands on exp. (Onsite: Juno Beach',
 'AWS Data Engineer - Splunk',
 'AWS Data Engineer Architect',
 'AWS Data Engineer III',
 'AWS Data Engineer Onshore_HartMap',
 'AWS Data Engineer(USC',
 'AWS Data Engineer--Minneapolis',
 'AWS Software Engineer III',
 'AWS and Databricks Data Engineer ',
 'AZURE DATA ENGINE

In [468]:
df['Easy Apply'].value_counts() / len(df)

Easy Apply
False    0.942369
True     0.057631
Name: count, dtype: float64

In [469]:
df_new['Easy Apply'].value_counts() / len(df_new)

Easy Apply
False    0.65554
True     0.34446
Name: count, dtype: float64

In [470]:
print(df.loc[df['Years Experience'].isna(),'Job Title'].shape[0])
df.loc[df['Years Experience'].isna(),'Job Title'].value_counts()

716


Job Title
Data Engineer                          137
Software Engineer                       25
Big Data Engineer                       21
Senior Data Engineer                    20
Sr. Data Engineer                        9
                                      ... 
Data and Analytics Engineer              1
Data Visualization Engineer              1
Data Systems Engineer/Informaticist      1
Data Storage Engineer Consultant         1
machine learning Engineer with AWS       1
Name: count, Length: 424, dtype: int64

In [471]:
print(df_new.loc[df_new['Years Experience'].isna(),'Job Title'].shape[0])
df_new.loc[df_new['Years Experience'].isna(),'Job Title'].value_counts()

493


Job Title
Data Engineer                                                     64
Senior Data Engineer                                              34
Data Engineer/Sr. Data Engineer (Remote) - EVOLVE Intelligence     8
Sr. Data Engineer                                                  7
Data Engineer (AWS, Azure, GCP)                                    6
                                                                  ..
Data Management/Systems Engineer                                   1
Data Management – ATLAS LCAT Systems Engineer 2                    1
Data Management Software Engineer                                  1
Data Management Engineer                                           1
data engineer lead                                                 1
Name: count, Length: 317, dtype: int64

In [472]:
print(list(df['Salary Estimate'].value_counts().index))

['60-109', '75-135', '82-130', '93-106', '94-164', '77-145', '67-72', '52-85', '108-127', '66-113', '73-94', '133-210', '54-104', '87-104', '54-103', '85-158', '105-121', '79-96', '94-166', '85-154', '108-199', '115-201', '79-146', '53-100', '84-113', '81-141', '164-192', '105-110', '57-75', '67-126', '57-108', '99-177', '95-99', '43-84', '74-134', '30-60', '113-198', '79-104', '69-122', '77-181', '56-108', '45-76', '65-120', '39-77', '88-136', '95-167', '94-107', '80-150', '191-217', '44-80', '27-50', '55-106', '54-113', '140-151', '71-130', '91-166', '52-63', '69-103', '110-188', '62-92', '55-105', '56-89', '44-141', '64-124', '65-101', '67-107', '74-93', '79-132', '88-163', '46-92', '100-160', '48-74', '28-53', '59-135', '73-126', '74-123', '106-127', '76-128', '154-281', '74-145', '49-98', '81-100', '100-181', '92-117', '66-122', '118-187', '70-138', '37-75', '57-92', '58-94', '57-97']


In [473]:
df_new['Salary Estimate'].value_counts()

Salary Estimate
160        25
100-200    21
85-170     16
90-115     16
-1         13
           ..
108-158     1
109-160     1
87-146      1
91-146      1
100-134     1
Name: count, Length: 1330, dtype: int64

In [474]:
pd.set_option('display.max_colwidth', 100)
df.loc[df['Job Title clean'].str.contains('\(') | df['Job Title clean'].str.contains('\)'), ['Job Title','Job Title clean']]

,Job Title,Job Title clean
2,(Big Data Developer) Applications Systems Engineer 4,(Big Data Developer) Applications Systems Engineer 4
3,"(Project Staff) Consultant, Data Engineer",(Project Staff) Consultant
4,"(SONUS) Engineer/Senior Engineer, IT Network Infrastructure",(SONUS) Engineer
5,(Senior) Data Engineer,(Senior) Data Engineer
8,"7232 Data Engineer (Analyst/Programmer - Career), Information Technology DevOps",7232 Data Engineer (Analyst
...,...,...
2444,Systems Engineer-(Ft. Worth or Wichita),Systems Engineer-(Ft. Worth or Wichita)
2450,Systems Integration Engineer (IO-ISR),Systems Integration Engineer (IO-ISR)
2454,TDM (Test Data Management) Engineer,TDM (Test Data Management) Engineer
2455,TDM (Test Data Management) Engineer with Mainframe exp,TDM (Test Data Management) Engineer - Mainframe exp


In [475]:
pd.set_option('display.max_colwidth', 100)
df_new.loc[df_new['Job Title clean'].str.contains('\(') | df_new['Job Title clean'].str.contains('\)'), ['Job Title','Job Title clean']]

,Job Title,Job Title clean
0,(ESEA) Systems Engineer level 2 (Data Flow and Routing Solutions) (TS/SCI w/FS Poly),(ESEA) Systems Engineer level 2 (Data Flow and Routing Solutions) (TS
1,(Hybrid) Senior Enterprise Data Engineer,(Hybrid) Senior Enterprise Data Engineer
2,(Hybrid) Senior Enterprise Data Engineer - San Antonio Texas,(Hybrid) Senior Enterprise Data Engineer
3,"(USA) Senior, Software Engineer - Data Ventures",(USA) Senior
5,"2025859 Data Scientist (Data Engineer) $215,000.00",2025859 Data Scientist (Data Engineer) $215
...,...,...
2404,"Systems Engineer (Data Recovery, Internet Trace Evidence Recovery & Analysis, Forensics Analysis...",Systems Engineer (Data Recovery
2405,Systems Engineer (HVAC),Systems Engineer (HVAC)
2416,Systems Engineer – Kirkland AFB (Data Sci/Arch Level 2),Systems Engineer – Kirkland AFB (Data Sci
2428,"Technical Support Engineer (L5) - Data Platform, Big Data / Analytics",Technical Support Engineer (L5)


In [476]:
df.loc[df['Job Title'].str.contains('/'), ['Job Title','Job Title clean']]

,Job Title,Job Title clean
4,"(SONUS) Engineer/Senior Engineer, IT Network Infrastructure",(SONUS) Engineer
8,"7232 Data Engineer (Analyst/Programmer - Career), Information Technology DevOps",7232 Data Engineer (Analyst
15,AI Researcher/Engineer - CV,AI Researcher
16,"AI/ML , Data Engineer",AI
17,"AI/ML ,Data Engineer",AI
...,...,...
2418,Systems Administrator III / Information Systems Security Engineer (ISSE),Systems Administrator III
2466,"Technical Leader, Software Engineer (C/C++, Python, Distributed Systems)",Technical Leader
2474,Test / DevSecOps Engineer,Test
2498,Validations Manager / Engineer,Validations Manager


In [477]:
pd.set_option('display.max_colwidth', 100)
df_new.loc[df_new['Job Title'].str.contains('/'), ['Job Title','Job Title clean']].head(55)

,Job Title,Job Title clean
0,(ESEA) Systems Engineer level 2 (Data Flow and Routing Solutions) (TS/SCI w/FS Poly),(ESEA) Systems Engineer level 2 (Data Flow and Routing Solutions) (TS
4,2024 Graduate - Data Scientist/Engineer - Decision Systems,2024 Graduate
6,5+ Years in VOICE/DATA COMM ENGINEER,5+ Years in VOICE
9,AI Engineer / Data Scientist,AI Engineer
10,AI Engineer/Data Scientist,AI Engineer
12,AI/ML Engineer-Data Scientist,AI
13,AI/ML Engineer-Data Scientist,AI
19,AWS / Data Engineer,AWS
34,AWS Data Engineer - Data Warehousing | Pipelining | End to End Processes - TS/SCI Req.,AWS Data Engineer
40,AWS Data Engineer(USC/GC Only),AWS Data Engineer(USC


In [478]:
pd.set_option('display.max_colwidth', 100)
df_new.loc[df_new['Job Title'].str.contains('/'), ['Job Title','Job Title clean']].tail(54)

,Job Title,Job Title clean
1977,Senior Network Engineer (VoIP/Data Comms),Senior Network Engineer (VoIP
1983,"Senior PowerBI Engineer and Data Analyst / Irving, TX",Senior PowerBI Engineer and Data Analyst
2002,Senior Software Engineer (Cloud/Data Lake ),Senior Software Engineer (Cloud
2022,Senior Software Engineer / Data Engineer - IDS1090,Senior Software Engineer
2031,"Senior Software Engineer, Content Services/Data",Senior Software Engineer
2042,"Senior Software Engineer, Data/ML (Full Stack)",Senior Software Engineer
2047,"Senior Software Engineer- Data Warehousing, ETL, Informatica, Python/Spark",Senior Software Engineer- Data Warehousing
2050,Senior Software Engineer/SRE- Data and Runtime Stability,Senior Software Engineer
2090,Senior System Engineer – (Bryan/College Station Data Center),Senior System Engineer – (Bryan
2093,Senior Systems Engineer (Data Analysis/Modeling & Simulation),Senior Systems Engineer (Data Analysis


In [479]:
df.groupby('Job Title clean').count()['Job Title'].sort_values(ascending=False).head(40)

Job Title clean
Data Engineer                     545
Senior Data Engineer              153
Software Engineer                 147
Big Data Engineer                  78
Senior Software Engineer           35
Systems Engineer                   17
Data Engineer II                   17
Data Scientist                     16
Machine Learning Engineer          16
Senior Big Data Engineer           16
Data Platform Engineer             14
Lead Data Engineer                 13
Data Warehouse Engineer            13
Azure Data Engineer                11
Federal                            10
Software Development Engineer      10
Cloud Engineer                      9
Full Stack Engineer                 9
DevOps Engineer                     9
Sales Engineer                      9
Data Analyst                        9
Business Intelligence Engineer      8
Engineer                            8
Senior Engineer                     8
Junior Data Engineer                8
DATA ENGINEER                     

In [480]:
df_new.groupby('Job Title clean').count()['Job Title'].sort_values(ascending=False).head(40)

Job Title clean
Data Engineer                                                383
Senior Data Engineer                                         228
Software Engineer                                             41
Senior Software Engineer                                      35
Lead Data Engineer                                            34
Principal Data Engineer                                       29
Lead Software Engineer                                        22
Senior Staff AI Data Engineer                                 22
Data Center Engineer                                          20
Azure Data Engineer                                           16
Data Engineer II                                              16
Data Integration Engineer                                     14
Staff Data Engineer                                           13
Staff Software Engineer                                       12
Senior Electrical Engineer                                    12
Systems E

In [481]:
set(df.columns)

{'Company Name',
 'Duplicate Count',
 'Easy Apply',
 'Founded',
 'Industry',
 'Job Description',
 'Job Title',
 'Job Title clean',
 'Location',
 'Rating',
 'Revenue',
 'Salary Estimate',
 'Salary Type',
 'Sector',
 'Size',
 'Type',
 'Years Experience',
 'job_description_cleaned',
 'max_salary',
 'min_salary'}

In [482]:
set(df_new.columns)

{'Company Name',
 'Duplicate Count',
 'Easy Apply',
 'Founded',
 'Industry',
 'Job Description',
 'Job Title',
 'Job Title clean',
 'Location',
 'Rating',
 'Revenue',
 'Salary Estimate',
 'Salary Type',
 'Sector',
 'Size',
 'Type',
 'Years Experience',
 'job_description_cleaned',
 'max_salary',
 'min_salary'}

In [483]:
set(df.columns) - set(df_new.columns)

set()

In [484]:
df.loc[:,'Type'].value_counts()

Type
Company - Private                 1387
Company - Public                   750
Subsidiary or Business Segment     125
Nonprofit Organization              48
Government                          21
College / University                12
Contract                            10
Private Practice / Firm              6
Other Organization                   4
Hospital                             3
Self-employed                        1
School / School District             1
Name: count, dtype: int64

In [485]:
df.loc[df['Salary Type']=='Per Hour']

,Job Title,Company Name,Location,Salary Estimate,Rating,Job Description,Founded,Industry,Revenue,Sector,Size,Type,Easy Apply,Duplicate Count,Job Title clean,Salary Type,min_salary,max_salary,job_description_cleaned,Years Experience
